In [3]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, DateType


In [4]:
spark = SparkSession.builder.appName('pyspark').getOrCreate()

FILENAME = '../temperature_data.txt'
schema = StructType([
    StructField('Day', DateType(), False),
    StructField('t', IntegerType(), False),
])

df = spark.read.csv(FILENAME, schema=schema)



In [7]:
new_df = df.select(F.year("Day").alias("year"), F.month("Day").alias("month"), "t")

grouped = new_df.groupBy(['year','month']).agg(F.mean('t').alias('avg_t'))
grouped.sort(['year','month'], ascending=True).dropna()

grouped.groupBy('year').agg(F.stddev(F.col('avg_t'))).dropna().sort(F.col('stddev_samp(avg_t)'), ascending=False).show(1)


+----+------------------+
|year|stddev_samp(avg_t)|
+----+------------------+
|2010| 12.57875744986771|
+----+------------------+
only showing top 1 row



In [8]:
grouped.groupBy('year').agg(F.stddev(F.col('avg_t'))).dropna().sort(F.col('stddev_samp(avg_t)'), ascending=False).show()


+----+------------------+
|year|stddev_samp(avg_t)|
+----+------------------+
|2010| 12.57875744986771|
|2011|10.634509176672408|
|2012| 10.63257898576577|
|2013| 10.37806618135188|
|2016|10.034330208723265|
|2014|   9.6816622392947|
|2009| 9.377698668187579|
|2015| 7.970702144434064|
|2008|7.7567808123431785|
+----+------------------+

